In [0]:
df = spark.read.csv("/Volumes/workspace/default/data/club_stats.csv", header=True, inferSchema=True)

df.show(5)
df.printSchema()


In [0]:
# Top goal Scorers
df.orderBy(df.Goals.desc()).select("Player","Club","Goals").show()

In [0]:
# Top Assist Providers

df.orderBy(df.Assists.desc()).select("Player","Club","Assists").show()

In [0]:
# Aggressive players with red and yellow cards

from pyspark.sql.functions import col, expr
df.withColumn("Total_Cards",col("Yellow_Cards") + col("Red_Cards")) \
    .orderBy(col("Total_cards").desc()) \
    .select("Player", "Club","Total_Cards") \
    .show()

In [0]:
# Club with most Goals

df.groupBy("Club").sum("Goals").orderBy("sum(Goals)", ascending=False).show()

In [0]:
#Player efficiency score
from pyspark.sql.functions import round

df.withColumn("Efficiency_Score",round((col("Goals")*4 + col("Assists") * 3 -col("Yellow_Cards") - col("Red_Cards") * 2) / col("Matches_Played"), 2)).orderBy("Efficiency_Score", ascending=False).select("Player", "Efficiency_Score").show()

In [0]:
#Club performnace summary

df.groupBy("Club").agg(
    {
        "Goals":"sum","Assists":"sum","Yellow_Cards":"sum","Red_cards":"sum","Matches_Played": "sum"
    }
).orderBy("sum(Goals)", ascending=False).show()

In [0]:
# Position Based stats

df.groupBy("Position").avg("Goals", "Assists", "Yellow_Cards", "Red_Cards").show()

In [0]:
#Top Consistent Players

df.withColumn("Goals_Per_Match", round(col("Goals") / col("Matches_Played"), 2)) \
    .orderBy("Goals_Per_match", ascending=False) \
    .select("Player", "Goals", "Matches_Played", "Goals_Per_Match") \
    .show()

In [0]:
#Cards per match ratio

df.withColumn("Aggressiveness_Score",
              round((col("Yellow_Cards") + col("Red_Cards"))  / col("Matches_Played"), 2)
    ).orderBy("Aggressiveness_score",ascending=False).select("Player", "Aggressiveness_Score").show()